# Missing Data

In [1]:
%pylab
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [9]:
from metric import tests
import cv2, tqdm
from utils import wand_transforms
import wand, random
from wand.image import Image
from metric.separate_iou import siou
from sklearn.metrics import adjusted_rand_score

In [3]:
#generate random images 100
!mkdir -p expr2_data/compare

In [11]:
for i in range(100):
    M = randint(25, 40)
    N = M - randint(5, 10, dtype=int)
    circs = tests.uniform_random_circles(M, 55, 450, 10, 50, 5)
    random.shuffle(circs)
    slices = np.zeros((N, 512, 512), np.uint8)
    image = np.zeros((512, 512), np.uint8)
    for j, (c, r) in enumerate(circs):
        if j < N:
            slices[j] = cv2.circle(slices[j], c, r, 1, -1)
        image = cv2.circle(image, c, r, 255, -1)
    cv2.imwrite(f'./expr2_data/image_{i}.png', image)
    np.save(f'./expr2_data/slices_{i}.npy', slices)

## Sanity check(with missing cells)

In [12]:
lst = ['niou', 'ari', 'f0_ari', 'f1_ari', 'ff_ari', 'siou_normal', 'siou_prop', 'siou_inv']
results = {k:[] for k in lst}

for i in tqdm.trange(100):
    slc = np.load(f'./expr2_data/slices_{i}.npy')
    img = cv2.imread(f'./expr2_data/image_{i}.png', cv2.IMREAD_ANYDEPTH)
    img = (img > 127).astype(uint8)
    for normalize in ['normal', 'prop', 'inv']:
        results['siou_'+normalize].append(siou(img, slc, normalize=normalize))
    
    orig = slc.sum(0).astype(uint8)
    tocompare = np.ones((512, 1026), uint8)
    tocompare[:, :512] = orig
    tocompare[:, 514:] = img
    cv2.imwrite(f'./expr2_data/compare/{i}.png', tocompare*255)
    
    _, x = cv2.connectedComponents(orig, connectivity=8)
    _, y = cv2.connectedComponents(img, connectivity=8)
    results['ari'].append(adjusted_rand_score(x.reshape(-1), y.reshape(-1)))
    
    idx = orig != 0
    results['f0_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    idx = img != 0
    results['f1_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    idx = (orig != 0) | (y != 0)
    results['ff_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    orig = orig.astype(bool)
    img = img.astype(bool)
    results['niou'].append((orig&img).sum()/(orig|img).sum())

for k in lst:
    ar = np.array(results[k])
    mean, std = ar.mean(), ar.std()
    print(f'{k:15s} {mean:.2f} {std:.2f}')
    
for k in lst:
    ar = np.array(results[k])
    mean, std = ar.mean(), ar.std()
    print(f'{mean:.2f} $\pm$ {std:.2f}', end=' & ')

100%|██████████| 100/100 [00:27<00:00,  3.72it/s]

niou            0.78 0.09
ari             0.82 0.08
f0_ari          1.00 0.00
f1_ari          0.74 0.16
ff_ari          0.74 0.16
siou_normal     1.00 0.00
siou_prop       1.00 0.00
siou_inv        1.00 0.00
0.78 $\pm$ 0.09 & 0.82 $\pm$ 0.08 & 1.00 $\pm$ 0.00 & 0.74 $\pm$ 0.16 & 0.74 $\pm$ 0.16 & 1.00 $\pm$ 0.00 & 1.00 $\pm$ 0.00 & 1.00 $\pm$ 0.00 & 

## Perturbed versions

In [13]:
tsfm = wand_transforms.Distort(4, 512, 10)
lst = ['niou', 'ari', 'f0_ari', 'f1_ari', 'ff_ari', 'siou_normal', 'siou_prop', 'siou_inv']
results = {k:[] for k in lst}

for i in tqdm.trange(100):
    img = Image(filename=f'./expr2_data/image_{i}.png')
    slc = np.load(f'./expr2_data/slices_{i}.npy')
    img, *_ = tsfm([img])
    img = np.frombuffer(img.make_blob('gray'), uint8).reshape(512,512)
    img = (img > 127).astype(uint8)
    for normalize in ['normal', 'prop', 'inv']:
        results['siou_'+normalize].append(siou(img, slc, normalize=normalize))
    
    orig = slc.sum(0).astype(uint8)
    tocompare = np.ones((512, 1026), uint8)
    tocompare[:, :512] = orig
    tocompare[:, 514:] = img
    cv2.imwrite(f'./expr2_data/compare/{i}.png', tocompare*255)
    
    _, x = cv2.connectedComponents(orig, connectivity=8)
    _, y = cv2.connectedComponents(img, connectivity=8)
    results['ari'].append(adjusted_rand_score(x.reshape(-1), y.reshape(-1)))
    
    idx = orig != 0
    results['f0_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    idx = img != 0
    results['f1_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    idx = (orig != 0) | (y != 0)
    results['ff_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    orig = orig.astype(bool)
    img = img.astype(bool)
    results['niou'].append((orig&img).sum()/(orig|img).sum())

for k in lst:
    ar = np.array(results[k])
    mean, std = ar.mean(), ar.std()
    print(f'{k:15s} {mean:.2f} {std:.2f}')
    
for k in lst:
    ar = np.array(results[k])
    mean, std = ar.mean(), ar.std()
    print(f'{mean:.2f} $\pm$ {std:.2f}', end=' & ')

100%|██████████| 100/100 [00:44<00:00,  2.16it/s]

niou            0.66 0.08
ari             0.70 0.07
f0_ari          0.85 0.05
f1_ari          0.52 0.13
ff_ari          0.48 0.11
siou_normal     0.93 0.04
siou_prop       0.98 0.01
siou_inv        0.87 0.07
0.66 $\pm$ 0.08 & 0.70 $\pm$ 0.07 & 0.85 $\pm$ 0.05 & 0.52 $\pm$ 0.13 & 0.48 $\pm$ 0.11 & 0.93 $\pm$ 0.04 & 0.98 $\pm$ 0.01 & 0.87 $\pm$ 0.07 & 